<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStardust/blob/main/Model_Agnostic_Meta_Learning_(MAML).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import copy

# Function to compute the loss for a given model and task
def compute_loss(model, task):
    inputs, targets = task
    outputs = model(inputs)
    loss = torch.nn.functional.mse_loss(outputs, targets)
    return loss

def maml_update(model, task_data, optimizer, lr_inner=0.01, inner_steps=5, lr_outer=0.001):
    # Outer loop: Update on meta-tasks
    for task in task_data:
        # Clone model for inner loop
        task_model = copy.deepcopy(model)

        # Inner loop: Task-specific update
        for step in range(inner_steps):
            task_loss = compute_loss(task_model, task)
            task_grads = torch.autograd.grad(task_loss, task_model.parameters(), create_graph=True)

            # Apply inner loop gradient update
            for param, grad in zip(task_model.parameters(), task_grads):
                param.data -= lr_inner * grad

        # Compute meta-loss and optimize on original model
        meta_loss = compute_loss(task_model, task)
        meta_loss.backward()

    optimizer.step()
    optimizer.zero_grad()

# Example usage
class SimpleModel(torch.nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = torch.nn.Linear(10, 1)

    def forward(self, x):
        return self.fc(x)

# Initialize model and optimizer
model = SimpleModel()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Generate some sample task data (inputs and targets)
task_data = [(torch.randn(32, 10), torch.randn(32, 1)) for _ in range(4)]

# Perform MAML update
maml_update(model, task_data, optimizer)

print("MAML update completed.")